# Visualize imagery over time 
In this notebook, we will show how to combine the [Data API](https://developers.planet.com/docs/apis/data/) and [Tiles Services API](https://developers.planet.com/docs/basemaps/tile-services/) to make a timelapse animation for a point of interest.

## Setup

#### Import Packages

In [ ]:
import json
import os
from datetime import datetime
import mercantile
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import requests

#### Set API Key
Get your API key from [your Planet account info](https://www.planet.com/account/#/) and paste it below in place of `YOUR_KEY_HERE`

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

## Data API Search
In this next part, we will search for items that match a given date range, item_type, and location

#### Data API Search

First, we need to define our AOI for our search

In [ ]:
# Convert coordinates to geojson geometry format
def coords_to_geometry(lat, lon):
    """Given latitude and longitude floats, construct a geojson geometry dict"""
    return {
        "type": "Point",
        "coordinates": [lon, lat]
    }

In [ ]:
# Make a geometry dictionary for coordinates in San Francisco

geom = coords_to_geometry(37.77493, -122.41942)
print(geom)

In [ ]:
lon = geom['coordinates'][0]
lat = geom['coordinates'][1]

Now let's define the other filters we want to use for our search:

In [ ]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

geom_filter = data_filter.geometry_filter(geom)
date_range_filter = data_filter.date_range_filter("acquired", datetime(month=1, day=1, year=2019), datetime(month=10, day=1, year=2019))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, date_range_filter, cloud_cover_filter])

Now we can run the search and view our item ids that result:

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.search(name='imagery_over_time',search_filter=combined_filter, item_types=item_types)
    item_list = [i async for i in items]

In [ ]:
for item in item_list:
    print(item['id'])

## Getting Webtiles
Although we could download images for the item_ids above, we can get a nice visual preview through [webtiles](https://developers.planet.com/docs/basemaps/tile-services/). These are 256x256 PNG images on a spatial grid, often used for [web maps](https://en.wikipedia.org/wiki/Tiled_web_map).


#### Generating tile urls
We want to get urls for many tiles over time for a given latitude, longitude, and zoom level.

Let's re-use some of the item_ids we found through the Data API.

In [ ]:
def get_tile_urls(lon, lat, item_list, limit=5, zoom=15,):
    """Get webtile urls for given coordinates, zoom, and matching filters.
    Args:
        lat: latitude float
        lon: longitude float
        zoom: zoom level int (usually between 1 and 15)
        limit: max number of ids to return
    Returns:
        tiles: json response and links to view tiles
    """
    item_list = item_list[:limit]
    tile = mercantile.tile(lon, lat, zoom)
    tile_url_template = 'https://tiles2.planet.com/data/v1/{item_type}/{item_id}/{z}/{x}/{y}.png?api_key={api_key}'
    return [tile_url_template.format(item_type=item_types, item_id=i, x=tile.x, y=tile.y, z=zoom, api_key=API_KEY) for i in item_list]




#### Testing tile urls
Click the links below to see tile images in your browser

In [ ]:
tile_urls = get_tile_urls(lon, lat, item_list, limit=5, zoom=15)
for url in tile_urls:
    print(url)
    print()

#### Display a tile

In [ ]:
from IPython.display import Image
resp = requests.get(tile_urls[0])
Image(resp.content)

#### Animate tiles over time

In [ ]:
%matplotlib inline
from IPython.display import HTML
import random
import time
def animate(urls, delay=1.0, loops=1):
    """Display an animated loop of images
    Args:
        urls: list of image url strings
        delay: how long in seconds to display each image
        loops: how many times to repeat the image sequence
    """
    disp_id = str(random.random())
    display("placeholder", display_id=disp_id)
    for loop in range(loops):
        for frame_url in urls:
            htmlDisplay = f'<img src="{frame_url}" class="mySlides">'
            display(HTML(htmlDisplay), display_id=disp_id, update=True)
            time.sleep(delay)

In [ ]:
animate(tile_urls, delay=0.5, loops=3)

In [ ]:
tile_urls = get_tile_urls(37.77493, -122.41942, item_list, limit=100)

In [ ]:
animate(tile_urls, delay=1, loops=3)